In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv("ıe423matchdata.csv", encoding='ISO-8859-1')

def calculate_control_limits(series, z=3):
    overall_mean = np.mean(series)
    std_dev = np.std(series)
    upper_limit = overall_mean + z * std_dev
    lower_limit = overall_mean - z * std_dev
    return overall_mean, upper_limit, lower_limit

# List of interesting matches to process
specified_fixture_ids = [19155147, 19155103, 19172041, 19155178, 19155143, 19172038, 19172095, 19155166]

for match_id in specified_fixture_ids:
    match_data = data[data['fixture_id'] == match_id]

    


    ball_possession_home = np.array(match_data['Ball.Possession_home'])
    ball_possession_away = np.array(match_data['Ball.Possession_away'])
    mean_home, _, _ = calculate_control_limits(ball_possession_home)
    mean_away, _, _ = calculate_control_limits(ball_possession_away)
    std_home = np.std(ball_possession_home)
    std_away = np.std(ball_possession_away)

    print(f"Match {match_id} - Sigma values:")
    print(f"Home team sigma: {std_home}, Away team sigma: {std_away}")

    # Detect events (difference minute by minute)
    event_columns = ['Redcards_home', 'Redcards_away', 'Goals_home', 'Goals_away']
    event_differences = match_data[event_columns].diff().fillna(0)  
    events = event_differences.sum(axis=1) 
    event_minutes = match_data[events > 0]['minute'].unique()  # Minutes where events occurred

    if len(event_minutes) == 0:
        print(f"Match {match_id}: No events detected.")
        continue

    print(f"Match {match_id} - Event minutes: {event_minutes}")

    # Check for 1.5 sigma mean shifts and calculate time intervals
    home_shift_detected = False
    away_shift_detected = False

    for minute in sorted(match_data['minute'].unique()):
        post_minute_data_home = match_data[match_data['minute'] >= minute]['Ball.Possession_home']
        post_minute_data_away = match_data[match_data['minute'] >= minute]['Ball.Possession_away']

        if not home_shift_detected and not post_minute_data_home.empty:
            post_minute_mean_home = post_minute_data_home.mean()
            if abs(post_minute_mean_home - mean_home) > (1.5 * std_home): #ADJUST SIGMA FRACTION 
                # Find the last event before the shift
                last_event_minute = max(event_minutes[event_minutes <= minute]) if len(event_minutes[event_minutes <= minute]) > 0 else None
                if last_event_minute is not None:
                    time_to_detect_home = minute - last_event_minute
                    print(f"Match {match_id} - Home team shift detected at minute {minute}. Last event at {last_event_minute}. Time interval: {time_to_detect_home} minutes.")
                home_shift_detected = True

        if not away_shift_detected and not post_minute_data_away.empty:
            post_minute_mean_away = post_minute_data_away.mean()
            if abs(post_minute_mean_away - mean_away) > (1.5 * std_away): #ADJUST SIGMA FRACTION 
                
                last_event_minute = max(event_minutes[event_minutes <= minute]) if len(event_minutes[event_minutes <= minute]) > 0 else None
                if last_event_minute is not None:
                    time_to_detect_away = minute - last_event_minute
                    print(f"Match {match_id} - Away team shift detected at minute {minute}. Last event at {last_event_minute}. Time interval: {time_to_detect_away} minutes.")
                away_shift_detected = True

        if home_shift_detected and away_shift_detected:
            break


Match 19155147 - Sigma values:
Home team sigma: 8.046147542703391, Away team sigma: 8.046147542703391
Match 19155147 - Event minutes: [ 6 10 15 29 34 42 59]
Match 19155103 - Sigma values:
Home team sigma: 11.02154661542192, Away team sigma: 11.02154661542192
Match 19155103 - Event minutes: [ 2 16 25 29 74]
Match 19172041 - Sigma values:
Home team sigma: 11.176938198252278, Away team sigma: 11.176938198252278
Match 19172041 - Event minutes: [10 18 30 37 48 63 81 92]
Match 19155178 - Sigma values:
Home team sigma: 11.219980865190605, Away team sigma: 11.197187251569
Match 19155178 - Event minutes: [15 27 39 46 58 74 94]
Match 19155143 - Sigma values:
Home team sigma: 11.708783796386331, Away team sigma: 11.708783796386331
Match 19155143 - Event minutes: [20 34 44 45 57 64 75]
Match 19172038 - Sigma values:
Home team sigma: 3.3628293753544454, Away team sigma: 3.3628293753544454
Match 19172038 - Event minutes: [ 2 41 43 65 67 75 83]
Match 19172038 - Home team shift detected at minute 91. 